# Model Development

In [26]:
%load_ext autoreload
%autoreload 2
p = print

import os

import pandas as pd

from sklearn.linear_model import LinearRegression, Lasso, MultiTaskLassoCV, Ridge, RidgeCV, MultiTaskElasticNet, MultiTaskElasticNetCV, ElasticNet, ElasticNetCV
from sklearn.metrics import mean_absolute_error, mean_squared_error

from crypr.util import get_project_path

import pickle

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [19]:
"""
Import Data.
"""
SYM = 'BTC'
Ty = 1
Tx = 72
MAX_LAG = 72
wavelet = 'haar_smooth'
data_dir = os.path.join(get_project_path(), 'data', 'processed')
models_dir = os.path.join(get_project_path(), 'models')

X_train = pd.read_csv(os.path.join(data_dir, 'X_train_{}_tx{}_ty{}_flag{}.csv'.format(SYM, Tx, Ty, MAX_LAG)))
Y_train = pd.read_csv(os.path.join(data_dir, 'y_train_{}_tx{}_ty{}_flag{}.csv'.format(SYM, Tx, Ty, MAX_LAG)))
X_test = pd.read_csv(os.path.join(data_dir, 'X_test_{}_tx{}_ty{}_flag{}.csv'.format(SYM, Tx, Ty, MAX_LAG)))
Y_test = pd.read_csv(os.path.join(data_dir, 'y_test_{}_tx{}_ty{}_flag{}.csv'.format(SYM, Tx, Ty, MAX_LAG)))

N_FEATURES = int(X_train.shape[1]/Tx)

p(X_train.shape)
X_train.head()

(5563, 1224)


,var1(t-72),var2(t-72),var3(t-72),var4(t-72),var5(t-72),var6(t-72),var7(t-72),var8(t-72),var9(t-72),var10(t-72),...,var8(t-1),var9(t-1),var10(t-1),var11(t-1),var12(t-1),var13(t-1),var14(t-1),var15(t-1),var16(t-1),var17(t-1)
0,13007702.37,1938.10,6677.90,6698.74,6674.83,6663.81,-0.045374,2.443024e+07,3635.216667,1.989071e+07,...,1.127572e+07,1713.281667,8.984999e+06,1362.789167,9.462649e+06,1433.027500,1.176674e+07,1789.942083,1.347986e+07,2042.300278
1,9378691.92,1399.77,6674.92,6688.86,6687.70,6671.52,0.192814,1.353682e+07,2016.126667,1.990098e+07,...,9.744978e+06,1481.570000,8.991674e+06,1364.466667,8.517821e+06,1289.817500,1.108382e+07,1684.964167,1.339104e+07,2029.368611
2,8424503.46,1254.90,6687.78,6696.86,6693.25,6686.64,0.082988,1.125509e+07,1677.520000,1.792497e+07,...,8.338396e+06,1266.540000,8.918546e+06,1353.785000,8.440196e+06,1278.243333,1.084223e+07,1647.623333,1.334990e+07,2023.472639
3,11276851.07,1681.92,6693.36,6697.32,6694.09,6673.00,0.012550,1.123341e+07,1674.920000,1.763622e+07,...,8.216904e+06,1250.025000,9.514413e+06,1445.685833,8.599417e+06,1303.322917,1.088818e+07,1654.636875,1.341289e+07,2033.546944
4,27926930.49,4202.29,6694.09,6697.69,6598.38,6562.33,-1.429769,1.342175e+07,2007.793333,1.922109e+07,...,7.977636e+06,1214.376667,9.550916e+06,1451.797500,8.613844e+06,1305.853333,1.086104e+07,1650.345417,1.335362e+07,2025.028333


# Let's try a simple linear regression model

In [18]:
lr_model = LinearRegression()
lr_model = lr_model.fit(X_train, Y_train)
lr_predict = lr_model.predict(X_test)

p(mean_absolute_error(y_pred=lr_predict, y_true=Y_test))
p(mean_squared_error(y_pred=lr_predict, y_true=Y_test))

0.35858397810378945
0.33283840653349356


In [21]:
# Save model
with open(os.path.join(models_dir, 'linear_model_{}.pkl'.format(SYM)), 'wb') as output_file:
    s = pickle.dump(lr_model, output_file)

# Other Linear Regression

## Lasso

In [23]:
lasso_params = {
    'alpha': [0.01],
}

lasso_model = Lasso(alpha=0.01, max_iter=10000)
lasso_model = lasso_model.fit(X=X_train, y=Y_train)

lasso_predict = lasso_model.predict(X_test)

p(mean_absolute_error(lasso_predict, Y_test))
p(mean_squared_error(lasso_predict, Y_test))

0.26849368264462337
0.25568177806059206


/Users/daniel_stevenson/anaconda3/envs/crypto_predict/lib/python3.5/site-packages/sklearn/linear_model/coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)


## Ridge


In [25]:
ridge_model = Ridge(alpha=0.01)
ridge_model = ridge_model.fit(X=X_train, y=Y_train)

ridge_predict = ridge_model.predict(X_test)

p(mean_absolute_error(ridge_predict, Y_test))
p(mean_squared_error(ridge_predict, Y_test))

0.2920772362272177
0.2816230918105076


## Elastic Net

In [27]:
enet_params = {
    'alpha': [1e-7],
}

enet_model = MultiTaskElasticNetCV(alphas=enet_params['alpha'])
enet_model = enet_model.fit(X=X_train, y=Y_train)

enet_predict = enet_model.predict(X_test)

p(mean_absolute_error(enet_predict, Y_test))
p(mean_squared_error(enet_predict, Y_test))

/Users/daniel_stevenson/anaconda3/envs/crypto_predict/lib/python3.5/site-packages/sklearn/model_selection/_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
/Users/daniel_stevenson/anaconda3/envs/crypto_predict/lib/python3.5/site-packages/sklearn/linear_model/coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/Users/daniel_stevenson/anaconda3/envs/crypto_predict/lib/python3.5/site-packages/sklearn/linear_model/coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/Users/daniel_stevenson/anaconda3/envs/crypto_pr

0.26252514468639
0.2532051706741615


/Users/daniel_stevenson/anaconda3/envs/crypto_predict/lib/python3.5/site-packages/sklearn/linear_model/coordinate_descent.py:1822: ConvergenceWarning: Objective did not converge, you might want to increase the number of iterations
  ConvergenceWarning)
